In [15]:
import geopandas as gpd
from shapely import box
import ee 
import geemap
import xee
import xarray as xr
from typing import Tuple
import dask

ee.Initialize()

In [16]:
gdf = gpd.read_file("data/Ethiopia_AdminBoundaries.shp").to_crs(4326)
study_regions = ["Tigray", "Amhara"]
study_area_gdf = gdf[gdf["R_NAME"].isin(study_regions)]

In [17]:
# Build uniform 1° grid for study area bounds
min_x, min_y, max_x, max_y = [round(coord) for coord in study_area_gdf.total_bounds.tolist()]
cell_size = 1
x_cells = int((max_x - min_x) / cell_size)
y_cells = int((max_y - min_y) / cell_size)

polygons = []
tile_ids = []
for i in range(x_cells):
    for j in range(y_cells):

        x1 = min_x + i * cell_size
        y1 = min_y + j * cell_size

        poly = box(x1, y1, x1 + cell_size, y1 + cell_size)
        tile_id = f"{x1}_{y1 + cell_size}"
        
        polygons.append(poly)
        tile_ids.append(tile_id)

tiles = gpd.GeoDataFrame({"tile_id": tile_ids, "geometry": polygons}, crs="EPSG:4326")

In [18]:
toi = tiles.iloc[27]

In [19]:
def gdf_to_ee_geometry(gdf_row):
    """Convert a GeoDataFrame row geometry to Earth Engine geometry"""
    return ee.Geometry.Polygon(gdf_row.geometry.__geo_interface__['coordinates'])

#### Landsat 8 (not used in this project)

In [5]:
def cloudmask_landsat(image):
    qa = image.select('QA_PIXEL')
    
    # Bit flags for Landsat-8 C2 QA_PIXEL
    fill_bit = 1 << 0        # Bit 0: Fill
    dilated_cloud = 1 << 1   # Bit 1: Dilated Cloud
    cirrus = 1 << 2          # Bit 2: Cirrus
    cloud = 1 << 3           # Bit 3: Cloud
    cloud_shadow = 1 << 4    # Bit 4: Cloud Shadow
    
    mask_bits = fill_bit | dilated_cloud | cirrus | cloud | cloud_shadow  # = 31
    qa_mask = qa.bitwiseAnd(mask_bits).eq(0)
    
    return image.updateMask(qa_mask)

def get_landsat8(geometry, start, end):
    """Load and preprocess Landsat-8 C2 imagery"""
    l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
            .filterDate(start, end)
            .filterBounds(geometry)
            .select(
                  ['B2', 'B3', 'B4','B5', 'B6', 'B7', 'QA_PIXEL'],
                  ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'QA_PIXEL']
            )
            .map(lambda img: (
                    img.addBands(img.normalizedDifference(['nir', 'red']).select(['nd'], ['NDVI']))
                )
            )
        )
    
    l8_cloud_masked = l8.map(cloudmask_landsat)
    
    return l8

In [6]:
def visualize_landsat_collection(collection, geometry):
    # Create the composite
    composite = collection.qualityMosaic('NDVI').clip(geometry).clip(geometry)

    # Create an interactive map
    Map = geemap.Map()
    Map.centerObject(tile_geom, zoom=11)

    # Add layers
    Map.addLayer(tile_geom, {'color': 'red'}, 'AOI')
    Map.addLayer(
        composite, 
        {'bands': ['red', 'green', 'blue'], 'min': 0, 'max': 0.3}, 
        'Landsat Composite'
    )

    # Display the map
    return Map

In [ ]:


# build collection and visualize
tile_geom = gdf_to_ee_geometry(toi)
start_2020 = ee.Date('2020-08-01')
end_2020 = ee.Date('2020-10-30')
start_2022 = ee.Date('2022-09-01')
end_2022 = ee.Date('2022-09-30')
l8_2020 = get_landsat8(tile_geom, start_2020, end_2020)
l8_2022 = get_landsat8(tile_geom, start_2022, end_2022)

### Sentinel-2

In [20]:
def add_cloud_shadow_mask(img, cloud_prob_thresh, nir_dark_thresh=0.15, cloud_proj_dist=1):
    """
    Add s2cloudless cloud and cloud shadow mask to Sentinel-2 image.
    
    Parameters:
    - cloud_prob_thresh: Cloud probability threshold (0-100). Default 50.
    - nir_dark_thresh: NIR reflectance threshold for identifying dark pixels (potential shadows).
    - cloud_proj_dist: Maximum distance (km) to search for cloud shadows from cloud edges.
    """
    # Get cloud probability
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    is_cloud = clouds.gte(cloud_prob_thresh)
    
    # Identify dark pixels (potential cloud shadows) using NIR
    dark_pixels = img.select('nir').lt(nir_dark_thresh)
    
    # Determine the direction to project cloud shadow from clouds
    # Use solar azimuth from image metadata
    shadow_azimuth = ee.Number(90).subtract(
        ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
    )
    
    # Project shadows from clouds
    cloud_proj = (is_cloud
                  .directionalDistanceTransform(shadow_azimuth, cloud_proj_dist * 10)
                  .reproject(crs=img.select('nir').projection(), scale=100)
                  .select('distance')
                  .mask()
                  .rename('cloud_transform'))
    
    # Identify cloud shadows
    shadows = cloud_proj.multiply(dark_pixels).rename('shadows')
    
    # Combine cloud and shadow mask
    is_cloud_or_shadow = is_cloud.add(shadows.gt(0)).gt(0)
    
    # Invert mask: 1 = clear, 0 = cloud/shadow
    is_clear = is_cloud_or_shadow.Not()
    
    return img.updateMask(is_clear)



def get_sentinel2(geometry, dates: Tuple, cloud_prob_thresh):
    """
    Load and preprocess Sentinel-2 imagery with s2cloudless masking.
    
    Parameters:
    - cloud_prob_thresh: Cloud probability threshold (0-100). Default 50 for conservative masking.
                        Use 65-75 for less aggressive masking in cloudy regions.
    """
    s2 = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterBounds(geometry)
        .filterDate(dates[0], dates[1])
        .map(lambda img: img
            .addBands(img.normalizedDifference(['B8', 'B4']).select(['nd'], ['NDVI']))
            .addBands(img.metadata('system:time_start'))
        )
        .map(lambda img: img
            .addBands(img.normalizedDifference(['B3', 'B8']).select(['nd'], ['NDWI']))
            .addBands(img.metadata('system:time_start'))
        )
        .map(lambda img: (
            img.select(['B2','B3','B4','B8','B11','B12'])
            .divide(10000)
            .addBands(img.select(['NDVI']))
            .addBands(img.select(['NDWI']))
            .copyProperties(img, ['system:time_start', 'MEAN_SOLAR_AZIMUTH_ANGLE'])
        ))
        .select(
            ['NDVI', 'NDWI','B2','B3','B4','B8','B11','B12'],
            ['NDVI', 'NDWI', 'blue','green','red','nir','swir1','swir2']
        )
    )

    s2Clouds = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                .filterBounds(geometry)
                .filterDate(dates[0], dates[1])
    )

    # Join S2 with cloud probability collection
    s2_with_clouds = ee.ImageCollection(ee.Join.saveFirst('cloud_mask').apply(
        primary=s2,
        secondary=s2Clouds,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    ))

    # Apply s2cloudless cloud and shadow masking
    s2_with_cloudless = s2_with_clouds.map(
        lambda img: add_cloud_shadow_mask(img, cloud_prob_thresh)
    )

    return ee.Image(s2_with_cloudless
                    .median()
                    .clip(geometry)
                    .set('system:time_start', ee.Date(dates[0]).millis())
            )

In [21]:
def visualize_sentinel2_collection(collection, geometry):
    #composite = collection.median()

    # Create an interactive map
    Map = geemap.Map()
    Map.centerObject(tile_geom, zoom=11)

    # Add layers
    Map.addLayer(tile_geom, {'color': 'red'}, 'AOI')
    Map.addLayer(
        collection, 
        {'bands': ['nir', 'red', 'green'], 'min': 0, 'max': 0.3}, 
        'Sentinel Composite'
    )

    # Display the map
    return Map

In [22]:
def generate_dates(year, season) -> Tuple:
    match season:
        case "bega":
            return (ee.Date(f'{year}-01-01'), ee.Date(f'{year}-03-31'))
        case "belg":
            return (ee.Date(f'{year}-04-01'), ee.Date(f'{year}-06-30'))
        case "kiremt":
            return (ee.Date(f'{year}-07-01'), ee.Date(f'{year}-09-30'))
        case "transition":
            return (ee.Date(f'{year}-10-01'), ee.Date(f'{year}-12-31')) 

In [23]:
tile_geom = gdf_to_ee_geometry(toi)
belg_20_dates = generate_dates(2020, "belg")
cloud_prob_thresh = 65
belg_20 = get_sentinel2(tile_geom, belg_20_dates, cloud_prob_thresh)

In [24]:
belg_20

In [26]:
visualize_sentinel2_collection(belg_20, toi)

Map(center=[12.500138500181183, 39.50000000000036], controls=(WidgetControl(options=['position', 'transparent_…

In [30]:
tile_geom = gdf_to_ee_geometry(toi)
kiremt_20_dates = generate_dates(2020, "kiremt")
cloud_prob_thresh = 65
belg_20 = get_sentinel2(tile_geom, belg_20_dates, cloud_prob_thresh)
kiremt_20 = get_sentinel2(tile_geom, kiremt_20_dates, cloud_prob_thresh)
visualize_sentinel2_collection(kiremt_20, toi)

Map(center=[12.500138500181183, 39.50000000000036], controls=(WidgetControl(options=['position', 'transparent_…

In [32]:
tile_geom = gdf_to_ee_geometry(toi)
kiremt_22_dates = generate_dates(2022, "kiremt")
cloud_prob_thresh = 65
kiremt_22 = get_sentinel2(tile_geom, kiremt_22_dates, cloud_prob_thresh)
visualize_sentinel2_collection(kiremt_22, toi)

Map(center=[12.500138500181183, 39.50000000000036], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
ds = xr.open_dataset(
    ee.ImageCollection([belg_20]),
    engine='ee',
    geometry=tile_geom,
    scale=10,
    crs="EPSG:32637",
    chunks={'lon': 1024, 'lat': 1024}  # Larger chunks for efficiency
)
ds

<xarray.Dataset> Size: 4GB
Dimensions:  (time: 1, X: 10845, Y: 11079)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-04-01
  * X        (X) float64 87kB 5e+05 5e+05 5e+05 ... 6.084e+05 6.084e+05
  * Y        (Y) float64 89kB 1.327e+06 1.327e+06 ... 1.437e+06 1.437e+06
Data variables:
    NDVI     (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
    NDWI     (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
    blue     (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
    green    (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
    red      (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
    nir      (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
    swir1    (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
    swir2    (time, X, Y) float32 481MB dask.array<chunksize=(1, 512, 256), meta=np.ndarray>
Attributes:
    crs:      EPSG:32637

In [12]:
def process_year(geom, year):
    """
    Process one tile for one year: create 4 seasonal median composites.
    
    Parameters:
    - geom: ee.Geometry for the tile
    - year: int (e.g., 2020)
    
    Returns:
    - xarray.Dataset
    """
    # Collect start and end dates per season
    bega_dates = generate_dates(year, "bega")
    belg_dates = generate_dates(year, "belg")
    kiremt_dates = generate_dates(year, "kiremt")
    transition_dates = generate_dates(year, "transition")

    # Generate S2 seasonal median composites
    cloud_prob_thresh = 65
    bega_img = get_sentinel2(geom, bega_dates, cloud_prob_thresh)
    belg_img = get_sentinel2(geom, belg_dates, cloud_prob_thresh)
    kiremt_img = get_sentinel2(geom, kiremt_dates, cloud_prob_thresh)
    transition_img = get_sentinel2(geom, transition_dates, cloud_prob_thresh)
    
    # Stack seasonal images into single multi-band image
    # Rename bands to include season suffix
    bega_renamed = bega_img.rename(['NDVI_bega', 'NDWI_bega', 'blue_bega', 'green_bega', 
                                     'red_bega', 'nir_bega', 'swir1_bega', 'swir2_bega'])
    belg_renamed = belg_img.rename(['NDVI_belg', 'NDWI_belg', 'blue_belg', 'green_belg', 
                                     'red_belg', 'nir_belg', 'swir1_belg', 'swir2_belg'])
    kiremt_renamed = kiremt_img.rename(['NDVI_kiremt', 'NDWI_kiremt', 'blue_kiremt', 'green_kiremt', 
                                         'red_kiremt', 'nir_kiremt', 'swir1_kiremt', 'swir2_kiremt'])
    transition_renamed = transition_img.rename(['NDVI_transition', 'NDWI_transition', 'blue_transition', 
                                                 'green_transition', 'red_transition', 'nir_transition', 
                                                 'swir1_transition', 'swir2_transition'])
    
    # Combine all seasons into single 32-band image (8 bands × 4 seasons)
    annual_composite = bega_renamed.addBands([belg_renamed, kiremt_renamed, transition_renamed])
    annual_composite = annual_composite.set({
        'tile_id': "39_12",
        'year': year,
        'system:time_start': ee.Date(f'{year}-01-01').millis()
    })
    
    # Create xarray dataset using xee with explicit spatial parameters
    ds = xr.open_dataset(
        ee.ImageCollection([annual_composite]),
        engine='ee',
        geometry=tile_geom,
        scale=10,
        crs="EPSG:32637",
    )
        
    return ds

In [13]:
data_2020 = process_year(tile_geom, 2020)
data_2020

<xarray.Dataset> Size: 15GB
Dimensions:           (time: 1, X: 10845, Y: 11079)
Coordinates:
  * time              (time) datetime64[ns] 8B 2020-01-01
  * X                 (X) float64 87kB 5e+05 5e+05 5e+05 ... 6.084e+05 6.084e+05
  * Y                 (Y) float64 89kB 1.327e+06 1.327e+06 ... 1.437e+06
Data variables: (12/32)
    NDVI_bega         (time, X, Y) float32 481MB ...
    NDWI_bega         (time, X, Y) float32 481MB ...
    blue_bega         (time, X, Y) float32 481MB ...
    green_bega        (time, X, Y) float32 481MB ...
    red_bega          (time, X, Y) float32 481MB ...
    nir_bega          (time, X, Y) float32 481MB ...
    ...                ...
    blue_transition   (time, X, Y) float32 481MB ...
    green_transition  (time, X, Y) float32 481MB ...
    red_transition    (time, X, Y) float32 481MB ...
    nir_transition    (time, X, Y) float32 481MB ...
    swir1_transition  (time, X, Y) float32 481MB ...
    swir2_transition  (time, X, Y) float32 481MB ...
Attributes:
    crs:      EPSG:32637

In [44]:
visualize_sentinel2_collection(s2_2020, tile_geom)

Map(center=[12.500138500181183, 39.50000000000036], controls=(WidgetControl(options=['position', 'transparent_…